# Connecting to AWS

In [15]:
# Use the ListFoundationModels API to show the models that are available in your region.
import boto3
             
# Create an &BR; client in the &region-us-east-1; Region.
bedrock = boto3.client(
    service_name="bedrock"
) 

# bedrock.list_foundation_models()
# bedrock.list_knowledge_bases() 

In [16]:
client = boto3.client('bedrock-agent')
response = client.list_knowledge_bases(
    maxResults=123
)
response

{'ResponseMetadata': {'RequestId': 'c9b47030-b93b-42d1-8720-13d8d358f3f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 30 Oct 2024 02:59:14 GMT',
   'content-type': 'application/json',
   'content-length': '160',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c9b47030-b93b-42d1-8720-13d8d358f3f8',
   'x-amz-apigw-id': 'AcYYfGZ4vHcEI5Q=',
   'x-amzn-trace-id': 'Root=1-6721a102-1be53aee79e3698a62fb0258'},
  'RetryAttempts': 0},
 'knowledgeBaseSummaries': [{'knowledgeBaseId': '1JORBFKUYS',
   'name': 'aerospace-chatbot-ams-demo',
   'status': 'ACTIVE',
   'updatedAt': datetime.datetime(2024, 10, 29, 17, 47, 24, 456721, tzinfo=tzutc())}]}

In [ ]:
kb_name='aerospace-chatbot-ams-demo'

# Pinecone database management

In [14]:
import os
from dotenv import load_dotenv
import pinecone

# Load environment variables from .env file
load_dotenv()

# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

index_name='aws-bedrock-ams-demo'

# Delete all vectors from each index but keep the index itself
index = pc.Index(index_name)
# Delete all vectors but preserve the index structure
try:
    index.delete(delete_all=True)
except Exception as e:
    print(f"Error deleting index (probably empty already): {e}")

Error deleting index (probably empty already): (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 30 Oct 2024 02:48:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '88', 'x-pinecone-request-id': '3920809686156093469', 'x-envoy-upstream-service-time': '89', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}



# Process PDFs

# Split PDF into chunks of 25 pages


In [ ]:
import os
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_path, pages_per_chunk=25):
    """Split a PDF into chunks of specified number of pages"""
    reader = PdfReader(input_path)
    total_pages = len(reader.pages)
    
    # Get base filename without extension
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    
    for chunk_start in range(0, total_pages, pages_per_chunk):
        writer = PdfWriter()
        chunk_end = min(chunk_start + pages_per_chunk, total_pages)
        
        # Add pages for this chunk
        for page_num in range(chunk_start, chunk_end):
            writer.add_page(reader.pages[page_num])
            
        # Save chunk to new PDF
        output_path = f"{base_name}_chunk_{chunk_start//pages_per_chunk + 1}.pdf"
        with open(output_path, "wb") as output_file:
            writer.write(output_file)

split_pdf('./AMS_1969_reocr.pdf')